In [396]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import re
import datetime
import requests

# Scraping Youtube

In [361]:
#스크래핑하는 함수
def crawling(query):
    driver = webdriver.Chrome()
    delay = 3
    driver.implicitly_wait(delay)

    url = 'https://www.youtube.com/results?search_query=' + query

    driver.get(url)
    driver.maximize_window()

    body = driver.find_element_by_tag_name('body')

    num_of_pagedowns = 20
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns -= 1

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    contents = soup.find_all('div','style-scope ytd-video-renderer')

    driver.close()
    
    return contents

In [464]:
#df에 insert하기 위한 list(row)를 반환하는 함수 
def insert_data(contents,query):
    for div in contents:
        try:
            #video_info table
            search_query = query
            title = div.find('yt-formatted-string','style-scope ytd-video-renderer').text
            contents = div.find('yt-formatted-string','metadata-snippet-text-navigation style-scope ytd-video-renderer').text
            video_meta = div.find_all('span','style-scope ytd-video-meta-block')
            view = video_meta[0].text
            create_date = video_meta[1].text
            video_url = div.find('a',"yt-simple-endpoint style-scope ytd-video-renderer")['href']
            
            #youtuber table
            youtuber_name = div.find('a','yt-simple-endpoint style-scope yt-formatted-string').text
            subscribers = ''
            channel_url = div.find('a','yt-simple-endpoint style-scope yt-formatted-string')['href']
            
            url = 'https://www.youtube.com'
            video.append([title, view, contents, create_date, search_query, url + video_url,youtuber_name])
            youtuber.append([youtuber_name,subscribers,url + channel_url])

        except:
            pass
        
    return video, youtuber

In [431]:
#구독자수 스크래핑하는 함수
def crawl_subscriber(url):
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
        }

    html = requests.get(url, headers=headers).text

    return re.findall("구독자(.+?)명", html)[-1]

# DF 정의 & Insert row

In [465]:
#youtubre_df, video_info_df 정의

youtube_columns = ['youtuber_name','subscribers','channel_url']
video_columns = ['title','view','contents','create_date','search_query','video_url','youtuber_name']

youtuber_df = pd.DataFrame(columns = youtube_columns)
video_info_df = pd.DataFrame(columns = video_columns)

In [466]:
#다음의 검색어를 이용해서 스크래핑을 진행한다. 
query_list = ['갤럭시', '갤럭시z플립 4', '갤럭시폴드4','아이폰', '아이폰14', 'iphone 14', 'iphone 14 pro']

#row 저장하는 리스트
video, youtuber = [],[]

for query in query_list:
    data = crawling(query)
    video, youtuber = insert_data(data,query)
    print('done! query',query,len(video),len(youtuber))

done! query 갤럭시 80 80
done! query 갤럭시z플립 4 156 156
done! query 갤럭시폴드4 220 220
done! query 아이폰 279 279
done! query 아이폰14 359 359
done! query iphone 14 436 436
done! query iphone 14 pro 463 463


In [467]:
#youtuber_df에 row 삽입
for row in youtuber:
    youtuber_df = youtuber_df.append(pd.Series(row,index = youtuber_df.columns),ignore_index = True)

#video_info_df에 row 삽입
for row in video:
    video_info_df = video_info_df.append(pd.Series(row,index = video_info_df.columns),ignore_index = True)

In [468]:
print(len(youtuber_df))
print(len(video_info_df))

463
463


In [469]:
#출력하기
youtuber_df.to_csv("[raw]youtuber_table.csv")
video_info_df.to_csv("[raw]video_info_table.csv")

## 전처리하기

In [470]:
youtuber_df.head()

,youtuber_name,subscribers,channel_url
0,ITSub잇섭,,https://www.youtube.com/c/ITSUB%EC%9E%87%EC%84%AD
1,주연 ZUYONI,,https://www.youtube.com/channel/UCB11SAf7WSN4G...
2,ITSub잇섭,,https://www.youtube.com/c/ITSUB%EC%9E%87%EC%84%AD
3,인태크 INTAECH,,https://www.youtube.com/channel/UCr6deoUPd2sh7...
4,고나고,,https://www.youtube.com/user/gonachoi


In [471]:
video_info_df.head()

,title,view,contents,create_date,search_query,video_url,youtuber_name
0,갤럭시 Z 플립4 언빡싱&달라진점 7가지! 플립3의 가장 큰 단점을 개선했다?,조회수 98만회,갬성의 삼성이라고 불렸던 플립3의 후속작인 플립4가 공개되었습니다. 이번에는 파격적...,1일 전,갤럭시,https://www.youtube.com/watch?v=WwGzNicmPHk,ITSub잇섭
1,미쳤다...너무 예쁜 플립4 산타마리아노벨라 언박싱 & 플립3와 달라진 점 총정리!...,조회수 14만회,인스타그램: https://www.instagram.com/zuyoni/ 오늘 언박...,5시간 전,갤럭시,https://www.youtube.com/watch?v=oi9qEieDaT8,주연 ZUYONI
2,1️⃣갤럭시 Z 플립4 & 폴드4 첫 인상! 궁금한게 많아서 개발자분께 모두 물어봤...,조회수 167만회,기다리고 기다렸던 삼성전자의 하반기 전략 플래그십 스마트폰. 갤럭시 Z 플립4와 폴...,6일 전,갤럭시,https://www.youtube.com/watch?v=H-wxJxEkmm4,ITSub잇섭
3,갤럭시워치5 프로 언박싱,조회수 722회,갤럭시워치5 프로 언박싱 영상 입니다. 디테일샷 및 사은품 소개가 포함되어 있습니다...,2시간 전,갤럭시,https://www.youtube.com/watch?v=NDWZ8x73ZjE,인태크 INTAECH
4,"갤럭시 Z 폴드4, 이게 갖고 다닐 만큼 유용할까?",조회수 1.9만회,"갤럭시 Z 폴드4 사전 예약하신 분 있나여!? 변화가 있는 듯 없는 듯 하지만, 힌...",5시간 전,갤럭시,https://www.youtube.com/watch?v=S7QFWqMnmmI,고나고


In [472]:
#view column 전처리하는 함수
def custom_view(x):
    try:
        if '만회' in x:
            m = 10000
        elif '천회' in x:
            m = 1000
        else:
            m = 1

        if '.' in x:
            f = re.findall("\d+.\d+", str(x))
            return int(float(f[0]) * m)

        if '.' not in x:        
            f = re.sub(r'[^0-9]', '', str(x))
            return int(f) * m
    
    except:
        return 0 

#date column 전처리하는 함수
def custom_date(x,current):
    num = int(re.sub(r'[^0-9]', '', str(x)))
    try:
        if '년 전' in x:
            date = current - datetime.timedelta(days=num*365)

        elif '개월 전' in x:
            date = current - datetime.timedelta(days=num*30)

        elif '주 전' in x:
            date = current - datetime.timedelta(days=num*7)

        elif '일 전' in x:
            date = current - datetime.timedelta(days=num)

        else:
            date = current
    except:
        date = current
    
    return date.strftime("%Y-%m-%d")

In [439]:
#구독자 수 전처리하는 함수
def custom_sub(x):
    if len(x) > 20:
        return 0 
        
    try:
        if '만' in x:
            m = 10000
        elif '천' in x:
            m = 1000
        else:
            m = 1

        if '.' in x:
            f = re.findall("\d+.\d+", str(x))
            return int(float(f[0]) * m)

        if '.' not in x:        
            f = re.sub(r'[^0-9]', '', str(x))
            return int(f) * m
    
    except:
        return 0 

In [473]:
#view column 전처리
video_info_df['view'] = video_info_df['view'].apply(lambda x : custom_view(x))

#create_date 전처리
current =  datetime.datetime.now()
video_info_df['create_date'] = video_info_df['create_date'].apply(lambda x: custom_date(x,current))

In [387]:
video_info_df.head()

,title,view,contents,create_date,search_query,video_url
0,갤럭시 Z 플립4 언빡싱&달라진점 7가지! 플립3의 가장 큰 단점을 개선했다?,970000,갬성의 삼성이라고 불렸던 플립3의 후속작인 플립4가 공개되었습니다. 이번에는 파격적...,2022-08-17,갤럭시,https://www.youtube.com/watch?v=WwGzNicmPHk
1,1️⃣갤럭시 Z 플립4 & 폴드4 첫 인상! 궁금한게 많아서 개발자분께 모두 물어봤...,1670000,기다리고 기다렸던 삼성전자의 하반기 전략 플래그십 스마트폰. 갤럭시 Z 플립4와 폴...,2022-08-17,갤럭시,https://www.youtube.com/watch?v=H-wxJxEkmm4
2,미쳤다...너무 예쁜 플립4 산타마리아노벨라 언박싱 & 플립3와 달라진 점 총정리!...,90000,인스타그램: https://www.instagram.com/zuyoni/ 오늘 언박...,2022-08-17,갤럭시,https://www.youtube.com/watch?v=oi9qEieDaT8
3,"갤럭시 Z 폴드4, 이게 갖고 다닐 만큼 유용할까?",13000,"갤럭시 Z 폴드4 사전 예약하신 분 있나여!? 변화가 있는 듯 없는 듯 하지만, 힌...",2022-08-17,갤럭시,https://www.youtube.com/watch?v=S7QFWqMnmmI
4,갤럭시워치5 프로 언박싱,415,갤럭시워치5 프로 언박싱 영상 입니다. 디테일샷 및 사은품 소개가 포함되어 있습니다...,2022-08-17,갤럭시,https://www.youtube.com/watch?v=NDWZ8x73ZjE


In [474]:
print(len(youtuber_df))

463


In [475]:
#youtuber_df에서 중복된 행을 제거한다.
youtuber_df=youtuber_df.drop_duplicates(keep='first')

In [394]:
#463 -> 185개로 줄어듦
print(len(youtuber_df))

185


## scraping subscribers using channel_url 

In [476]:
dic = {}

for url in youtuber_df['channel_url']:
    if '...' in url:
        continue
    
    data = crawl_subscriber(url)
    print(url, data)
    dic[url] = data

https://www.youtube.com/c/ITSUB%EC%9E%87%EC%84%AD  219만
https://www.youtube.com/channel/UCB11SAf7WSN4GrCquKoOHrw  59만
https://www.youtube.com/channel/UCr6deoUPd2sh7mafAk5xWDQ  3.41천
https://www.youtube.com/user/gonachoi  13.1만
https://www.youtube.com/c/SamsungKorea  44.1만
https://www.youtube.com/channel/UC2mFPcCVDdQLL6WyQ0pqRaA  2
https://www.youtube.com/c/Samsung  622만
https://www.youtube.com/c/%EC%82%AC%EC%95%BC%EB%90%98%EB%82%98Ten  8.8천
https://www.youtube.com/c/TheSINZA  10.6만
https://www.youtube.com/c/jerrylreny  6.45만
https://www.youtube.com/c/%EC%97%90%EC%8A%A4%EC%98%A4%EB%94%94STORY  58.7만
https://www.youtube.com/c/binigoon79  4.73만
https://www.youtube.com/c/tablet79  3.44천
https://www.youtube.com/channel/UCsqJJPqihIU25fCNiwR5Thg  6.11만
https://www.youtube.com/c/%EA%B3%A0%EB%8B%88TV-GONITV  5.91만
https://www.youtube.com/channel/UCJKZoVf3RIMfFffjdseqdEg  38.7만
https://www.youtube.com/channel/UCFX6adXoyQKxft933NB3rmA  60.3만
https://www.youtube.com/c/%EC%9D%B8%EC%8A%A4%ED%8E%99%E

https://www.youtube.com/c/Dave2D  357만
https://www.youtube.com/c/MaxTechOfficial  100만
https://www.youtube.com/c/XEETECHCARE  167만
https://www.youtube.com/c/jojol67  233만
https://www.youtube.com/c/TotallyStudios  60.1만
https://www.youtube.com/c/BrandonButch  90.4만
https://www.youtube.com/c/YourAppleTuber  15만
https://www.youtube.com/c/AppleAdvice  2.59만
https://www.youtube.com/channel/UCSQEQ7QojWLZ6EQJSpJ02tQ  1.35만
https://www.youtube.com/c/9to5mac  75.6만
https://www.youtube.com/c/neshudo  103만
https://www.youtube.com/c/AppleExplained  128만
https://www.youtube.com/c/EverythingApplePro  827만
https://www.youtube.com/c/DemonsTech  6.34천
https://www.youtube.com/c/TechReviewer1  3.28만
https://www.youtube.com/c/MiniStudiosFK  27만
https://www.youtube.com/c/hollywoodfrodo  2.87만
https://www.youtube.com/c/GregsGadgets  23.4만
https://www.youtube.com/c/TheAppleCircle  25.6만
https://www.youtube.com/c/TechnicalGuruji  2230만
https://www.youtube.com/c/GigaContent1  5.58천
https://www.youtube.com/c/El

In [478]:
#join하기 위한 df 선언
join = pd.DataFrame(dic.items(),columns= ['channel_url','subscribers'])

#subscribers 칼럼 전처리
join['subscribers'] = join['subscribers'].apply(lambda x: custom_sub(x))

In [481]:
#youtuber_df에 subscribers 칼럼을 추가
new = pd.merge(youtuber_df,join,on = 'channel_url')

#youtuber_df 총 row 확인
len(new)

164

In [ ]:
#최종 youtuber_df와 video_info_df 출력
new.to_csv("[complete]youtuber_df.csv")
video_info_df.to_csv("[complete]video_info_df.csv")

# Table에 넣기 위한 DF 전처리

In [ ]:
video = pd.read_csv('[complete]video_info_df.csv',index_col = 0)
youtuber = pd.read_csv('[complete]youtuber_df.csv',index_col = 0)

In [ ]:
data = pd.merge(video, youtuber,on = 'youtuber_name')

data = data.rename(columns = {'id':'youtuber_id'})
data =data.drop('subscribers', axis=1)
data =data.drop('channel_url', axis=1)
data =data.drop('youtuber_name', axis=1)

data.columns

youtuber = youtuber[['id', 'youtuber_name', 'subscribers','channel_url']]
data = data[['title', 'view', 'contents', 'create_date', 'search_query', 'video_url','youtuber_id']]

In [ ]:
#txt가 들어있는 칼럼 전체에 특수 문자를 제거한다. 
youtuber['youtuber_name']= youtuber['youtuber_name'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)
data['title'] = data['title'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)
data['contents'] = data['contents'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

In [ ]:
#최종 df 출력
youtuber.to_csv('[final]youtuber_df.csv')
data.to_csv('[final]video_info_df.csv')